# Analysis of the Toronto area

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. 

Again, a different notebook is asked, so the code below will not be explain and serves only the purpose to get the dataframe.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pgeocode
nomi = pgeocode.Nominatim('CA')
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(source, 'lxml')
data=[]
header=[]
table= soup.find('table',attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    head = row.find_all('th')
    cols = [ele.text.strip() for ele in cols]
    heads= [ele.text.strip() for ele in head]
    data.append([ele for ele in cols if ele]) # Get rid of empty values
    header.append([ele for ele in heads if ele])
del(data[0]) # supprimer la ligne de départ
header= list(filter(None,header))
df = pd.DataFrame(data, columns=header[0])
deleted='Not assigned'
for row in df.itertuples():    
    if row.Neighbourhood == deleted:
        df.Neighbourhood.iloc[row.Index] = df.Borough.iloc[row.Index]
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df['latitude']=""
df['longitude']=""
nomi = pgeocode.Nominatim('CA')
for lieu in df.itertuples():
    pcode =  'XXX ' + lieu.Postcode
    location = nomi.query_postal_code(pcode)
    df.ix[lieu.Index,'longitude']=location.longitude
    df.ix[lieu.Index, 'latitude']=location.latitude    
df.head(2)

C:\Users\Yann\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Yann\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,Postcode,Borough,Neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8113,-79.193
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7878,-79.1564


## Toronto area
Let us first select the area where 'Toronto' is mentioned:

In [2]:
df_Toronto = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
print('There are',df_Toronto.shape[0],'Borough namely related to Toronto')

There are 38 Borough namely related to Toronto


## Initialisation of the notebook

In [3]:
CLIENT_ID = 'AJBQSLBUMRWPSQWCYSVAJBGN2JL01KK0UPM5WXIEWF1QHK' # modified Foursquare ID
CLIENT_SECRET = 'F5OTF0XD5NPZVHLZXPCRXZEXE4LQDZEVBD4NK3LNROF242' # modified Foursquare Secret
VERSION = '20180604'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules (for the map)
import matplotlib.cm as cm
import matplotlib.colors as colors
# import Folium
import folium 

print('Libraries imported.')

LIMIT=100
radius=500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

print('Functions defined')

Libraries imported.
Functions defined


# Cluster analysis of the Toronto area

In [4]:
Toronto_venues = getNearbyVenues(names=df_Toronto['Postcode'],
                                   latitudes=df_Toronto['latitude'],
                                   longitudes=df_Toronto['longitude']
                                  )


In [5]:
Toronto_venues.groupby('Neighborhood').count().shape

(37, 6)

One borough is missing, namely M5N:

In [6]:
df_Toronto.loc[df_Toronto['Postcode']=='M5N']

,Postcode,Borough,Neighbourhood,latitude,longitude
22,M5N,Central Toronto,Roselawn,43.7113,-79.4195


Looking 'manually' at this specific entry one find the reason:

In [7]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    43.7113, 
    -79.4195, 
    radius, 
    LIMIT)
requests.get(url).json()

{'meta': {'code': 200, 'requestId': '5cdac23a351e3d1d4e9da4d3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Lawrence Park South',
  'headerFullLocation': 'Lawrence Park South, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.715800004500004,
    'lng': -79.41328609487343},
   'sw': {'lat': 43.7067999955, 'lng': -79.42571390512657}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

Which is that there is not enough results. Let's get rid of this one and move on !

In [8]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 218 uniques categories.


Encoding the differents categories:

In [9]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']
T2=Toronto_onehot.groupby('Neighborhood').sum()
#Toronto_onehot.groupby('Neighborhood').mean()

Now, I will use a different approach than the one seen in class to cluster the neighborood. Instead of looking at the main venue in each postal code, I will compare the frequency of the category with respect to an average neighborood. It is close to the chi-squared method, as we will compare the expected frequency of a venue to the observed frequency.

In [10]:
# first I define the expected frequency of each venue 
temp= np.zeros(shape=(len(T2.index),len(T2.columns)))
df_one = pd.DataFrame(temp, columns=T2.columns, index=T2.index)
#df_one[:]=T2
df_one[:]=int(1)
TT=T2-df_one.mul(T2.sum(axis=1),axis=0).mul(T2.sum(axis=0),axis=1).div(T2.sum(axis=0).sum())

In [11]:
num_top_venues = 10

Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head(1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Bakery,Health Food Store,Pub,Gastropub,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [12]:
# create a new dataframe
n_venues_sorted = pd.DataFrame(columns=columns)
n_venues_sorted['Neighborhood'] = TT.index

for ind in np.arange(TT.shape[0]):
    n_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TT.iloc[ind, :], num_top_venues)

n_venues_sorted.head(1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Pub,Gastropub,Bakery,Pool,Lake,History Museum,Hobby Shop,Women's Store,Hookah Bar


We can see the difference between the two approaches. "Bakery" is the most common venue in this area but the "Health Food store" is more common <b>relatively to the other postcode</b>.

The cluster analysis will now be conducted:

In [13]:
# set number of clusters
kclusters = 5

TT_clustering = TT #.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TT_clustering)

# add clustering labels
n_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = pd.concat([df_Toronto.set_index('Postcode'),
                            n_venues_sorted.set_index('Neighborhood')],
                           axis=1,join='inner')
Toronto_merged # check the last columns!

,Borough,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M4E,East Toronto,The Beaches,43.6784,-79.2941,0,Health Food Store,Pub,Gastropub,Bakery,Pool,Lake,History Museum,Hobby Shop,Women's Store,Hookah Bar
M4K,East Toronto,"The Danforth West,Riverdale",43.6803,-79.3538,0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Fruit & Vegetable Store,Trail,Juice Bar,Brewery,Yoga Studio,Indian Restaurant,Spa
M4L,East Toronto,"The Beaches West,India Bazaar",43.6693,-79.3155,0,Sandwich Place,Park,Light Rail Station,Pet Store,Fish & Chips Shop,Movie Theater,Brewery,Food & Drink Shop,Burrito Place,Liquor Store
M4M,East Toronto,Studio District,43.6561,-79.3406,0,Baseball Field,Garden Center,Coworking Space,Performing Arts Venue,Gym,Park,Coffee Shop,Home Service,Lake,Hobby Shop
M4N,Central Toronto,Lawrence Park,43.7301,-79.3935,0,Photography Studio,Park,Home Service,Lake,History Museum,Hobby Shop,Women's Store,Hookah Bar,Hostel,Indie Movie Theater
M4P,Central Toronto,Davisville North,43.7135,-79.3887,0,Dog Run,Food & Drink Shop,Clothing Store,Gym,Breakfast Spot,Park,Jewelry Store,Hawaiian Restaurant,Health Food Store,History Museum
M4R,Central Toronto,North Toronto West,43.7143,-79.4065,0,Gym Pool,Garden,Playground,Park,Home Service,Lake,Hobby Shop,Women's Store,Hookah Bar,Hostel
M4S,Central Toronto,Davisville,43.702,-79.3853,0,Dessert Shop,Sandwich Place,Italian Restaurant,Toy / Game Store,Farmers Market,Indian Restaurant,Fast Food Restaurant,Diner,Café,Thai Restaurant
M4T,Central Toronto,"Moore Park,Summerhill East",43.6899,-79.3853,0,Tennis Court,Playground,Grocery Store,Thai Restaurant,Gym,Park,Health Food Store,History Museum,Hobby Shop,Women's Store
M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.6861,-79.4025,0,Light Rail Station,Coffee Shop,Convenience Store,Supermarket,Home Service,Health Food Store,History Museum,Hobby Shop,Women's Store,Hookah Bar


In [14]:
#define the center coodinates:
[latitude,longitude]=Toronto_merged[['latitude','longitude']].mean()

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['latitude'], Toronto_merged['longitude'], Toronto_merged.index, Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters